In [89]:
import numpy as np
import pandas as pd
import re

In [31]:
pd_master = pd.read_csv("A_product_master_2.csv")
pd_master.head()

,Product_ID,Client_ID,Product_Code,Product_Desc,UOM,Packsize,Manufacturer,MRP,PTS,PTR,...,Category,molecular_id,P_ID,GST_PER,Manufacturer_Code,ED_MDM_Product_Code,ED_MDM_Product_Image,ED_MDM_Product_Document_Image,ED_MDM_Product_Creation_Time,Match
0,1697255,756,J0002111,A MAX CAPSULES 10^S,NaN,10'S,DERMAWIZ LAB DERMA,155.0,108.50,108.50,...,NaN,NaN,117107987.0,NaN,1h,ED000157592,NaN,NaN,NaN,NaN
1,1696011,756,J0002112,A PLUS TABLETS,NaN,NaN,MEDLIFE GENETICS-LAM,230.0,0.00,155.90,...,NaN,NaN,NaN,NaN,GXN,ED000148859,NaN,NaN,NaN,NaN
2,1714647,756,J0002113,A TO Z GOLD CAPSULES NS 15CAPSULES,NaN,15CAP,ALKEM #M CARE#,210.0,142.37,142.39,...,NaN,NaN,117111597.0,NaN,BB,ED000000174,NaN,NaN,NaN,NaN
3,1686756,756,J0002115,A TO Z GOLD CAPSULES 15^S,NaN,15'S,ALKEM ALTIS,210.0,128.14,142.37,...,NaN,NaN,117124974.0,NaN,GXQ,ED000155268,NaN,NaN,NaN,NaN
4,7790857,756,A00849,A TO Z IMMUNE TABLETS 1*16,NaN,1*16,ALKEM #M CARE#,215.0,145.76,145.76,...,NaN,NaN,118559972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
variable = pd_master['Product_Desc']
word = 'TABLETS'
my_list = []
for i in variable:
    if i.find(word) != -1:
        i = i.split(word)
        j =  word + i[-1]
        k = [i[0], j]
        my_list.append(k)
    else:
        k = [i]
        my_list.append(k)

# print(my_list)

In [48]:
dfT = pd.DataFrame(pd_master['Product_Desc'])

p1T = []
p2T = []

for p in my_list:
    if len(p) == 2:
        p1T.append(p[0])
        p2T.append(p[-1])
    elif len(p) == 1:
        p1T.append(p[0])
        p2T.append([])
        
dfT["First_part"] = p1T
dfT["Type"] = p2T
dfT['Type'] = dfT['Type'].apply(lambda x: np.nan if len(x) == 0 else x)


In [49]:
dfT.head()

,Product_Desc,First_part,Type
0,A MAX CAPSULES 10^S,A MAX CAPSULES 10^S,NaN
1,A PLUS TABLETS,A PLUS,TABLETS
2,A TO Z GOLD CAPSULES NS 15CAPSULES,A TO Z GOLD CAPSULES NS 15CAPSULES,NaN
3,A TO Z GOLD CAPSULES 15^S,A TO Z GOLD CAPSULES 15^S,NaN
4,A TO Z IMMUNE TABLETS 1*16,A TO Z IMMUNE,TABLETS 1*16


In [35]:
def replace_pattern(string):
    pattern = r'(\d+)\^S'
    
    def replace_digit(digit_match):
        digit = int(digit_match.group(1))
        return f'Strip of {digit}'
    
    modified_string = re.sub(pattern, replace_digit, string)
    
    return modified_string
    

In [91]:
variable_2 = pd_master['Product_Desc']
split_words = ["CAPSULES", "TABLETS", "GEL","SYRUP","POWDER", "OINTMENT","INJECTION.","DROP","CREAM", "SOAP", "VACCINE"]
my_list1 = []
for i in variable_2:
    match_counter = 0
    if "^S" in i:
        i = replace_pattern(i)
    #if i.find("CAPSULES", "TABLETS") != -1:
    for x in split_words:
        if x in i:
            i = i.split(x)
            j =  x + i[-1]
            k = [i[0], j]
            my_list1.append(k)
            match_counter = 1
    if match_counter == 0:
        k = [i]
        my_list1.append(k)
        
# print(my_list1)

In [92]:
df_new = pd.DataFrame(pd_master["Product_Desc"])

list_p_1 = []
list_p_2 = []

for elements in my_list1:
    if len(elements) == 2:
        list_p_1.append(elements[0])
        list_p_2.append(elements[-1])
    else:
        list_p_1.append(elements[0])
        list_p_2.append(np.nan)

df_new["First_part"] = list_p_1
df_new["Type"] = list_p_2

In [93]:
df_new['First_part'] = df_new['First_part'].str.replace(" ", "")

In [94]:
df_new.head()

,Product_Desc,First_part,Type
0,A MAX CAPSULES 10^S,AMAX,CAPSULES Strip of 10
1,A PLUS TABLETS,APLUS,TABLETS
2,A TO Z GOLD CAPSULES NS 15CAPSULES,ATOZGOLD,CAPSULES
3,A TO Z GOLD CAPSULES 15^S,ATOZGOLD,CAPSULES Strip of 15
4,A TO Z IMMUNE TABLETS 1*16,ATOZIMMUNE,TABLETS 1*16


In [88]:
df_new.to_csv("File.csv")